In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
!ls

Auction Challenge.ipynb
Causality and Endogeneity.ipynb
Gen Data Exp Re.ipynb
Gen Data Exp Re_1_1.ipynb
Gen Data Exp.ipynb
Gen Data Exp_1_1.ipynb
Gen Data.ipynb
Gen_Data_Re .ipynb
Modeling and Cross Validation-Copy2.ipynb
Modeling and Cross Validation-Exp.ipynb
Modeling and Cross Validation.ipynb
X_exp_test.csv
X_exp_train.csv
X_test.csv
X_train.csv
app.py
app_2.py
app_3.py
auction_feature_engineering.ipynb
auction_feature_engineering_Exp.ipynb
auction_feature_engineering_Exp_1_1.ipynb
auction_wrangling.ipynb
data
data_1.csv
data_10.csv
data_2.csv
data_3.csv
data_4.csv
data_5.csv
data_6.csv
data_7.csv
data_8.csv
data_9.csv
data_exp_1.csv
data_exp_1_1.csv
data_exp_1_2.csv
data_exp_2.csv
heatmap.jpg
historical_data1_2017
setup.sql
strat_two_data_1.csv
strat_two_data_10.csv
strat_two_data_2.csv
strat_two_data_3.csv
strat_two_data_4.csv
strat_two_data_5.csv
strat_two_data_6.csv
strat_two_data_7.csv
strat_two_data_8.csv
strat_two_data_9.csv
y_exp_test.csv
y_exp_train.csv
y_test.csv
y_train.

## Import Data

#### Strat one is a dataframe containing data generated using a "true value" bidding strategy. Strat two is a  dataframe containg data identical to strat one *except* for bidding strategy, which is 90% of true value.

In [3]:
strat_one = ["data_1.csv", "data_2.csv", "data_3.csv", "data_4.csv", "data_5.csv",
            "data_6.csv", "data_7.csv", "data_8.csv", "data_9.csv", "data_10.csv", 
             "data_exp_1.csv", "data_exp_1_1.csv" ]
strat_two = ["strat_two_data_1.csv", "strat_two_data_2.csv", "strat_two_data_3.csv", 
            "strat_two_data_4.csv", "strat_two_data_5.csv", "strat_two_data_6.csv",
            "strat_two_data_7.csv", "strat_two_data_8.csv", "strat_two_data_9.csv",
            "strat_two_data_10.csv", "data_exp_2.csv", "data_exp_1_2.csv"]

df_one = pd.DataFrame()
for strat in strat_one:
    df_one = df_one.append(pd.read_csv(strat))
df_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008 entries, 0 to 143
Data columns (total 22 columns):
Unnamed: 0      1008 non-null int64
auction type    1008 non-null object
count           1008 non-null float64
mean            1008 non-null float64
std             1008 non-null float64
min             1008 non-null float64
25              1008 non-null float64
50              1008 non-null float64
75              1008 non-null float64
max             1008 non-null float64
count_per       1008 non-null float64
mean_per        1008 non-null float64
std_per         1008 non-null float64
min_per         1008 non-null float64
25_per          1008 non-null float64
50_per          1008 non-null float64
75_per          1008 non-null float64
max_per         1008 non-null float64
skew            1008 non-null float64
kurtosis        1008 non-null float64
4th             1008 non-null float64
5th             1008 non-null float64
dtypes: float64(20), int64(1), object(1)
memory usage: 181.1+

In [4]:
df_two = pd.DataFrame()
for strat in strat_two:
    df_two = df_two.append(pd.read_csv(strat))
df_two.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008 entries, 0 to 143
Data columns (total 22 columns):
Unnamed: 0      1008 non-null int64
auction type    1008 non-null object
count           1008 non-null float64
mean            1008 non-null float64
std             1008 non-null float64
min             1008 non-null float64
25              1008 non-null float64
50              1008 non-null float64
75              1008 non-null float64
max             1008 non-null float64
count_per       1008 non-null float64
mean_per        1008 non-null float64
std_per         1008 non-null float64
min_per         1008 non-null float64
25_per          1008 non-null float64
50_per          1008 non-null float64
75_per          1008 non-null float64
max_per         1008 non-null float64
skew            1008 non-null float64
kurtosis        1008 non-null float64
4th             1008 non-null float64
5th             1008 non-null float64
dtypes: float64(20), int64(1), object(1)
memory usage: 181.1+

In [5]:
df_one.head(1)

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,std_per,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th
0,0,second price,1986.0,933.164149,924.064666,2.0,293.0,650.5,1268.0,6492.0,...,0.122761,0.000247,0.051906,0.111822,0.20501,0.833935,1.489933,3.055051,0.001371,0.000604


In [6]:
#Add indicator variable for whether an observation was generated from a distribution of second
#price auction returns.

def second_price(auction_type):
    if auction_type == "second price":
        return 1
    else:
        return 0
    

df_one['second_price'] = df_one['auction type'].apply(lambda x: second_price(x))
df_one.head()

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price
0,0,second price,1986.0,933.164149,924.064666,2.0,293.0,650.5,1268.00,6492.0,...,0.000247,0.051906,0.111822,0.205010,0.833935,1.489933,3.055051,0.001371,0.000604,1
1,1,second price,1940.0,940.464433,926.662232,2.0,292.0,647.0,1278.75,7545.0,...,0.000297,0.052843,0.119329,0.206110,0.812251,1.402719,2.594558,0.001175,0.000472,1
2,2,second price,1987.0,940.079517,898.821615,1.0,302.0,662.0,1331.00,7697.0,...,0.000283,0.054402,0.117069,0.198419,0.774154,1.358119,2.306657,0.001011,0.000380,1
3,3,second price,1919.0,954.547681,916.283537,1.0,283.5,684.0,1337.00,6627.0,...,0.000174,0.053173,0.120087,0.209268,0.884148,1.373355,2.723853,0.001185,0.000508,1
4,4,second price,1931.0,958.780425,922.844696,1.0,310.5,689.0,1291.00,7282.0,...,0.000115,0.055359,0.122503,0.210825,0.791184,1.363539,2.307352,0.001229,0.000484,1


In [7]:
#Add indicator variable for whether an observation was generated from a distribution of second
#price auction returns.
df_two['second_price'] = df_two['auction type'].apply(lambda x: second_price(x))
df_two.head()

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price
0,0,second price,1056.0,1409.969697,826.684968,174.0,798.0,1204.5,1835.25,5785.0,...,0.100250,0.142131,0.191375,0.263837,0.707386,1.409730,2.483590,0.000497,0.000158,1
1,1,second price,1069.0,1424.252572,857.175929,188.0,842.0,1231.0,1767.00,5567.0,...,0.100165,0.142832,0.191009,0.265977,0.802526,1.576014,3.018403,0.000735,0.000263,1
2,2,second price,1111.0,1454.627363,863.563722,220.0,841.5,1232.0,1872.50,5274.0,...,0.100343,0.143243,0.193661,0.276721,0.848952,1.501194,3.089941,0.000808,0.000317,1
3,3,second price,1087.0,1419.823367,877.972860,244.0,808.0,1203.0,1790.50,6678.0,...,0.100172,0.138480,0.185577,0.267518,0.849028,1.617007,3.291350,0.000932,0.000372,1
4,4,second price,1058.0,1484.840265,925.614038,300.0,841.0,1227.5,1921.75,5832.0,...,0.100286,0.142149,0.198711,0.277442,0.701531,1.444662,2.442999,0.000781,0.000265,1


In [8]:
#EDA
df_two[df_two['auction type'] == "variable percent"].head()

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price
45,45,variable percent,1086.0,922.450276,343.196252,263.0,645.25,903.0,1165.00,2048.0,...,0.117741,0.127000,0.136138,0.153995,0.163212,0.011326,-1.237789,9.164670e-08,3.144189e-11,0
46,46,variable percent,1050.0,912.647619,342.597825,274.0,640.00,874.5,1172.75,1998.0,...,0.117778,0.126991,0.144822,0.154013,0.163205,-0.014757,-1.324233,1.045362e-07,-5.914670e-11,0
47,47,variable percent,1058.0,918.214556,343.851052,212.0,650.25,896.5,1152.00,2236.0,...,0.117754,0.127003,0.144848,0.153978,0.163208,-0.009832,-1.192618,9.039892e-08,-2.944614e-12,0
48,48,variable percent,1098.0,919.425319,331.318174,305.0,645.25,901.5,1158.25,2024.0,...,0.117772,0.127003,0.144882,0.154006,0.163343,-0.034359,-1.262277,9.812753e-08,-1.193927e-10,0
49,49,variable percent,1074.0,915.359404,338.413589,227.0,641.25,873.5,1171.75,1993.0,...,0.117817,0.127017,0.144857,0.153995,0.163164,-0.017219,-1.244499,9.267186e-08,-6.780410e-11,0


In [9]:
#EDA
df_one[df_one['auction type'] == "variable percent"].head()

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price
45,45,variable percent,1906.0,284.873033,154.493380,40.0,161.25,255.5,371.75,877.0,...,0.019845,0.030000,0.040088,0.059996,0.070148,0.037190,-1.266568,1.467547e-07,2.048593e-10,0
46,46,variable percent,1928.0,278.805498,147.700060,36.0,166.75,251.0,367.25,874.0,...,0.019802,0.029996,0.040067,0.059986,0.070204,0.024555,-1.235626,1.418833e-07,1.249752e-10,0
47,47,variable percent,1915.0,282.368668,151.994256,48.0,165.00,246.0,375.50,837.0,...,0.019837,0.029988,0.040087,0.060000,0.070162,0.030817,-1.268499,1.464049e-07,1.861423e-10,0
48,48,variable percent,1902.0,279.505783,147.849667,41.0,163.25,249.0,372.75,808.0,...,0.019802,0.029992,0.040083,0.059992,0.070153,0.041303,-1.239978,1.424284e-07,2.354571e-10,0
49,49,variable percent,1939.0,284.918515,152.060232,40.0,169.50,254.0,371.50,942.0,...,0.019827,0.030006,0.049883,0.060003,0.070148,-0.015238,-1.264302,1.455770e-07,-9.429159e-11,0


In [10]:
#EDA
df_one[df_one['auction type'] == "mixed strategy"].head()

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price
63,63,mixed strategy,1875.0,344.048533,652.411530,0.0,0.0,92.0,383.50,5899.0,...,0.0,0.0,0.019963,0.061320,0.715498,2.897573,10.894456,0.000766,0.000350,0
64,64,mixed strategy,1978.0,348.138524,639.001695,0.0,0.0,99.0,403.75,6530.0,...,0.0,0.0,0.019978,0.060042,0.695498,2.748498,9.343176,0.000646,0.000273,0
65,65,mixed strategy,1968.0,351.517276,646.200549,0.0,0.0,128.0,406.00,6950.0,...,0.0,0.0,0.021036,0.068616,0.763571,2.872701,10.579134,0.000955,0.000458,0
66,66,mixed strategy,1910.0,340.448168,615.082998,0.0,0.0,102.0,407.50,6564.0,...,0.0,0.0,0.019968,0.060061,0.561313,2.519919,7.294094,0.000496,0.000177,0
67,67,mixed strategy,1956.0,357.805215,683.976780,0.0,0.0,113.0,388.00,7404.0,...,0.0,0.0,0.020022,0.062530,0.716054,2.691399,8.461557,0.000847,0.000360,0


In [11]:
#EDA
df_two[df_two['auction type'] == "mixed strategy"].head()

,Unnamed: 0,auction type,count,mean,std,min,25,50,75,max,...,min_per,25_per,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price
63,63,mixed strategy,1189.0,931.588730,598.243726,206.0,561.00,795.0,1076.00,5706.0,...,0.099868,0.100000,0.110525,0.153977,0.634299,2.889785,9.910659,0.000400,0.000143,0
64,64,mixed strategy,1079.0,927.350324,613.291616,211.0,555.00,779.0,1063.50,5766.0,...,0.099811,0.100000,0.102722,0.150419,0.656757,3.087306,12.051387,0.000370,0.000140,0
65,65,mixed strategy,1092.0,936.515568,610.884793,203.0,552.75,790.0,1086.25,5844.0,...,0.099842,0.100000,0.117958,0.153980,0.749784,3.039459,12.329490,0.000570,0.000256,0
66,66,mixed strategy,1041.0,943.501441,603.464184,227.0,562.00,799.0,1094.00,5480.0,...,0.099824,0.100000,0.117840,0.154037,0.702417,2.962176,11.131293,0.000454,0.000178,0
67,67,mixed strategy,1152.0,962.540799,669.101933,213.0,561.75,785.5,1127.00,5579.0,...,0.099844,0.100009,0.117995,0.154007,0.660089,3.000446,11.340946,0.000446,0.000171,0


## Feature Engineering

In [12]:
df_one['total auction returns'] = df_one['count'] * df_one['mean']
df_two['total auction returns'] = df_two['count'] * df_one['mean']

In [13]:
df_one['std/mean'] = df_one['std'] / df_one['mean']
df_two['std/mean'] = df_two['std'] / df_two['mean']

In [21]:
df_two = df_two.reset_index()

In [22]:
df_two.head()

,index,Unnamed: 0,auction type,count,mean,std,min,25,50,75,...,50_per,75_per,max_per,skew,kurtosis,4th,5th,second_price,total auction returns,std/mean
0,0,0,second price,1056.0,1409.969697,826.684968,174.0,798.0,1204.5,1835.25,...,0.191375,0.263837,0.707386,1.409730,2.483590,0.000497,0.000158,1,9.854213e+05,0.586314
1,1,1,second price,1069.0,1424.252572,857.175929,188.0,842.0,1231.0,1767.00,...,0.191009,0.265977,0.802526,1.576014,3.018403,0.000735,0.000263,1,1.005356e+06,0.601843
2,2,2,second price,1111.0,1454.627363,863.563722,220.0,841.5,1232.0,1872.50,...,0.193661,0.276721,0.848952,1.501194,3.089941,0.000808,0.000317,1,1.044428e+06,0.593667
3,3,3,second price,1087.0,1419.823367,877.972860,244.0,808.0,1203.0,1790.50,...,0.185577,0.267518,0.849028,1.617007,3.291350,0.000932,0.000372,1,1.037593e+06,0.618368
4,4,4,second price,1058.0,1484.840265,925.614038,300.0,841.0,1227.5,1921.75,...,0.198711,0.277442,0.701531,1.444662,2.442999,0.000781,0.000265,1,1.014390e+06,0.623376


## Concat dataframes

In [ ]:
#Concat dataframes to create one observation for each row containing data for both 
#bidding strategies.


In [26]:
new_columns_one = []
for column in df_one.columns:
    new_columns_one.append(column +"_1")
df_one.columns = new_columns_one

new_columns_two = []
for column in df_two.columns:
    new_columns_two.append(column +"_2")
df_two.columns = new_columns_two

result_df = pd.concat([df_one,df_two], axis=1, sort = True)
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008 entries, 0 to 1007
Data columns (total 52 columns):
index_1                    1008 non-null int64
Unnamed: 0_1               1008 non-null int64
auction type_1             1008 non-null object
count_1                    1008 non-null float64
mean_1                     1008 non-null float64
std_1                      1008 non-null float64
min_1                      1008 non-null float64
25_1                       1008 non-null float64
50_1                       1008 non-null float64
75_1                       1008 non-null float64
max_1                      1008 non-null float64
count_per_1                1008 non-null float64
mean_per_1                 1008 non-null float64
std_per_1                  1008 non-null float64
min_per_1                  1008 non-null float64
25_per_1                   1008 non-null float64
50_per_1                   1008 non-null float64
75_per_1                   1008 non-null float64
max_per_1       

In [27]:
result_df.head()

,index_1,Unnamed: 0_1,auction type_1,count_1,mean_1,std_1,min_1,25_1,50_1,75_1,...,50_per_2,75_per_2,max_per_2,skew_2,kurtosis_2,4th_2,5th_2,second_price_2,total auction returns_2,std/mean_2
0,0,0,second price,1986.0,933.164149,924.064666,2.0,293.0,650.5,1268.00,...,0.191375,0.263837,0.707386,1.409730,2.483590,0.000497,0.000158,1,9.854213e+05,0.586314
1,1,1,second price,1940.0,940.464433,926.662232,2.0,292.0,647.0,1278.75,...,0.191009,0.265977,0.802526,1.576014,3.018403,0.000735,0.000263,1,1.005356e+06,0.601843
2,2,2,second price,1987.0,940.079517,898.821615,1.0,302.0,662.0,1331.00,...,0.193661,0.276721,0.848952,1.501194,3.089941,0.000808,0.000317,1,1.044428e+06,0.593667
3,3,3,second price,1919.0,954.547681,916.283537,1.0,283.5,684.0,1337.00,...,0.185577,0.267518,0.849028,1.617007,3.291350,0.000932,0.000372,1,1.037593e+06,0.618368
4,4,4,second price,1931.0,958.780425,922.844696,1.0,310.5,689.0,1291.00,...,0.198711,0.277442,0.701531,1.444662,2.442999,0.000781,0.000265,1,1.014390e+06,0.623376


## Feature Engineering: Create ratios of strat1 : strat2

In [28]:
#Creates ratios of total returns under alternate bidding strategies, as well as ratios of
#standard deviations and maxes

result_df['total return ratio'] = result_df['total auction returns_1']/ result_df['total auction returns_2']
result_df['ratio stdev'] = result_df['std_1']/result_df['std_2']
result_df['max ratio'] = result_df['max_1']/result_df['max_2']

In [29]:
result_df.head()

,index_1,Unnamed: 0_1,auction type_1,count_1,mean_1,std_1,min_1,25_1,50_1,75_1,...,skew_2,kurtosis_2,4th_2,5th_2,second_price_2,total auction returns_2,std/mean_2,total return ratio,ratio stdev,max ratio
0,0,0,second price,1986.0,933.164149,924.064666,2.0,293.0,650.5,1268.00,...,1.409730,2.483590,0.000497,0.000158,1,9.854213e+05,0.586314,1.880682,1.117795,1.122213
1,1,1,second price,1940.0,940.464433,926.662232,2.0,292.0,647.0,1278.75,...,1.576014,3.018403,0.000735,0.000263,1,1.005356e+06,0.601843,1.814780,1.081064,1.355308
2,2,2,second price,1987.0,940.079517,898.821615,1.0,302.0,662.0,1331.00,...,1.501194,3.089941,0.000808,0.000317,1,1.044428e+06,0.593667,1.788479,1.040828,1.459424
3,3,3,second price,1919.0,954.547681,916.283537,1.0,283.5,684.0,1337.00,...,1.617007,3.291350,0.000932,0.000372,1,1.037593e+06,0.618368,1.765409,1.043635,0.992363
4,4,4,second price,1931.0,958.780425,922.844696,1.0,310.5,689.0,1291.00,...,1.444662,2.442999,0.000781,0.000265,1,1.014390e+06,0.623376,1.825142,0.997008,1.248628


In [32]:
result_df = result_df.drop(columns = [ 'index_1', 'Unnamed: 0_1', 'count_1', 'mean_1',
       'std_1', 'min_1', '25_1', '50_1', '75_1', 'max_1', 'count_per_1',
       'mean_per_1', 'std_per_1', 'min_per_1', 'total auction returns_1', 'index_2',
       'Unnamed: 0_2', 'auction type_2', 'count_2', 'mean_2', 'std_2', 'min_2',
       '25_2', '50_2', '75_2', 'max_2', 'count_per_2', 'mean_per_2',
       'std_per_2', 'min_per_2', '25_per_2', '50_per_2', '75_per_2',
       'max_per_2', 'second_price_2',
       'total auction returns_2'])
result_df.columns

Index(['auction type_1', '25_per_1', '50_per_1', '75_per_1', 'max_per_1',
       'skew_1', 'kurtosis_1', '4th_1', '5th_1', 'second_price_1',
       'std/mean_1', 'skew_2', 'kurtosis_2', '4th_2', '5th_2', 'std/mean_2',
       'total return ratio', 'ratio stdev', 'max ratio'],
      dtype='object')

In [34]:
X = result_df.drop(columns = ["auction type_1", "second_price_1"])
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008 entries, 0 to 1007
Data columns (total 17 columns):
25_per_1              1008 non-null float64
50_per_1              1008 non-null float64
75_per_1              1008 non-null float64
max_per_1             1008 non-null float64
skew_1                1008 non-null float64
kurtosis_1            1008 non-null float64
4th_1                 1008 non-null float64
5th_1                 1008 non-null float64
std/mean_1            1008 non-null float64
skew_2                1008 non-null float64
kurtosis_2            1008 non-null float64
4th_2                 1008 non-null float64
5th_2                 1008 non-null float64
std/mean_2            1008 non-null float64
total return ratio    1008 non-null float64
ratio stdev           1008 non-null float64
max ratio             1008 non-null float64
dtypes: float64(17)
memory usage: 141.8 KB


## Scale data 

In [35]:
min_max_scaler = preprocessing.MinMaxScaler()
X_minmax = min_max_scaler.fit_transform(X)
X_minmax

array([[0.51926314, 0.63544457, 0.65232389, ..., 0.13905502, 0.31137727,
        0.36302529],
       [0.52863583, 0.67810035, 0.65582187, ..., 0.1286495 , 0.29998074,
        0.4466249 ],
       [0.54423356, 0.66525936, 0.6313514 , ..., 0.12449666, 0.28749683,
        0.4839659 ],
       ...,
       [0.        , 0.1268549 , 0.14326872, ..., 0.25      , 0.14692034,
        0.13245226],
       [0.        , 0.22757954, 0.2329243 , ..., 0.07894737, 0.2840664 ,
        0.27652817],
       [0.        , 0.17069333, 0.19074185, ..., 0.23684211, 0.38868678,
        0.40462214]])

In [36]:
y = result_df.drop(columns= ['25_per_1', '50_per_1', '75_per_1', 'max_per_1',
       'skew_1', 'kurtosis_1', '4th_1', '5th_1', 
       'std/mean_1', 'skew_2', 'kurtosis_2', '4th_2', '5th_2', 'std/mean_2',
       'total return ratio', 'ratio stdev', 'max ratio'])
y.head()

,auction type_1,second_price_1
0,second price,1
1,second price,1
2,second price,1
3,second price,1
4,second price,1


In [37]:
def ordinal(auction):
    if auction == "second price":
        return 0
    elif auction == "constant percent":
        return 1
    elif auction == "variable percent":
        return 2
    elif auction == "soft floor":
        return 3
    elif auction == "mixed strategy":
        return 4
y['auction type'] = y['auction type_1'].apply(lambda x: ordinal(x))
y.head()

,auction type_1,second_price_1,auction type
0,second price,1,0
1,second price,1,0
2,second price,1,0
3,second price,1,0
4,second price,1,0


In [38]:
y = y.drop(columns=['auction type_1'])
y.head()

,second_price_1,auction type
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


## Split data into train and test

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [40]:
X_train.to_csv("X_exp_1_train.csv", sep = ",")
X_test.to_csv("X_exp_1_test.csv", sep = ",")
y_train.to_csv("y_exp_1_train.csv", sep = ",")
y_test.to_csv("y_exp_1_test.csv", sep = ",")

In [41]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008 entries, 0 to 1007
Data columns (total 2 columns):
second_price_1    1008 non-null int64
auction type      1008 non-null int64
dtypes: int64(2)
memory usage: 23.6 KB
